In [2]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import numpy as np
from matplotlib import pyplot as plt
from torchvision import models
from torch import nn
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, f1_score
import copy
from torch.optim import lr_scheduler # 스케줄러 임포트

In [3]:
# 기존 하이퍼파라미터 및 데이터 로딩 설정 (동일)
HYPERPARAMS = {'batch_size' : 32, 'learning_rate' : 0.001, 'epochs' : 10, # 에포크를 좀 더 길게 설정 (조기 종료 있으므로)
			   'transform' : transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
												 transforms.Normalize(mean=[0.48235, 0.45882, 0.40784], std=[0.229, 0.224, 0.225])])}

In [4]:
# 데이터셋 경로 확인 및 클래스 이름 가져오기
# TODO: 데이터셋 경로 확인 필요
try:
    TRAIN_DATASET = ImageFolder('./data/train', transform = HYPERPARAMS['transform'])
    VAILD_DATASET = ImageFolder('./data/vaildation', transform = HYPERPARAMS['transform']) # 'vaildation' -> 'validation' 오타 수정
    class_names = TRAIN_DATASET.classes
    num_classes = len(class_names)
    print(f"데이터셋에서 클래스 이름 {class_names} ({num_classes}개) 을(를) 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"데이터셋 로딩 또는 클래스 이름 불러오는 데 실패했습니다. 오류: {e}")
    print("데이터셋 경로('./data/train', './data/validation')를 확인하거나, 수동으로 클래스 개수를 지정하세요.")
    # TODO: 실패 시 수동으로 클래스 개수 지정 필요
    num_classes = 3 # 예시로 3개 클래스 가정
    class_names = [f'class_{i}' for i in range(num_classes)]
    print(f"임시 클래스 개수 {num_classes}개와 임시 클래스 이름 {class_names} 을(를) 사용합니다.")

데이터셋에서 클래스 이름 ['cheetah_train_resized', 'hyena_train_resized', 'tiger_train_resized'] (3개) 을(를) 성공적으로 불러왔습니다.


In [5]:
TRAIN_DATALOADER = DataLoader(TRAIN_DATASET, batch_size=HYPERPARAMS['batch_size'], shuffle=True, drop_last=True)
VAILD_DATALOADER = DataLoader(VAILD_DATASET, batch_size=HYPERPARAMS['batch_size'], shuffle=True, drop_last=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # 사용 가능한 경우 GPU, 아니면 CPU 사용

# 손실 함수는 모델 종류와 무관하게 동일하게 사용
criterion = nn.CrossEntropyLoss().to(DEVICE)

In [6]:
# --- 학습할 VGG 모델 목록 정의 ---
models_to_train = {
    'VGG11': models.vgg11, # 모델 생성 함수
    'VGG13': models.vgg13,
    'VGG16': models.vgg16,
    'VGG19': models.vgg19,
}
# 각 모델에 맞는 사전 학습 가중치 Enum
model_weights = {
    'VGG11': models.VGG11_Weights.IMAGENET1K_V1,
    'VGG13': models.VGG13_Weights.IMAGENET1K_V1,
    'VGG16': models.VGG16_Weights.IMAGENET1K_V1,
    'VGG19': models.VGG19_Weights.IMAGENET1K_V1,
}

In [8]:
# VGG 모델 반복문 시작
for model_name, model_fn in models_to_train.items():
    print(f"\n--- {model_name} 모델 학습 시작 ---")

    # 1. 모델 로드 (사전 학습 가중치 포함) 및 최종 분류기 수정
    try:
        weights = model_weights[model_name]
        model = model_fn(weights=weights)

        # VGG 모델의 classifier 마지막 레이어 수정 (VGG11, 13, 16, 19 모두 동일하게 classifier[6] 수정)
        # VGG 구조에 따라 이 인덱스가 다를 수 있지만, 일반적으로 6번입니다.
        if isinstance(model.classifier[6], nn.Linear):
             # 최종 레이어 입력 특징 수를 가져옴
             in_features = model.classifier[6].in_features
             model.classifier[6] = nn.Linear(in_features, num_classes)
             print(f"{model_name} 모델의 최종 분류 레이어를 {in_features} -> {num_classes} 으로 수정했습니다.")
        else:
             print(f"경고: {model_name} 모델의 classifier[6]이 예상한 Linear 레이어가 아닙니다. 구조를 확인하세요.")
             # 만약 VGG 구조가 다르면 이 부분을 수정해야 합니다.
             # 예: print(model.classifier) 를 통해 구조 확인

        model = model.to(DEVICE) # 모델을 디바이스로 이동

    except Exception as e:
        print(f"오류: {model_name} 모델 로드 또는 수정 중 오류 발생: {e}. 이 모델 학습을 건너뜝니다.")
        continue # 다음 모델로 이동

    # 2. Optimizer 및 Scheduler 생성 (현재 모델에 맞춰 새로 생성)
    optimizer = optim.Adam(model.parameters(), lr=HYPERPARAMS['learning_rate'])

    # 스케줄러 정의 (현재 Optimizer에 맞춰 새로 생성)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                               mode='min',
                                               factor=0.1,
                                               patience=5, # 검증 손실이 5 에포크 동안 개선되지 않으면 학습률 감소
                                               verbose=True)


    # 3. 조기 종료 및 모델 저장을 위한 변수 초기화 (현재 모델 학습을 위해 새로 초기화)
    best_val_loss = float('inf') # 검증 손실 추적
    best_val_accuracy = 0.0     # 최고 정확도 추적 (최저 손실 시점의 정확도)
    patience = 3 # 검증 손실이 개선되지 않아도 기다릴 에포크 수 (스케줄러와 별개)
    patience_counter = 0 # 검증 손실이 개선되지 않은 에포크 수 카운터
    best_model_state = None # 가장 좋은 성능을 보인 모델의 state_dict 저장 변수
    best_epoch = -1 # 가장 좋은 성능을 보인 에포크 기록

    print(f"{model_name} 모델 학습 시작 (최대 {HYPERPARAMS['epochs']} 에포크, patience={patience})")

    # 4. 학습 루프 시작
    for epoch in range(HYPERPARAMS['epochs']):
        # 학습 단계
        model.train() # 학습 모드
        train_cost = 0.0
        for images, classes in tqdm(TRAIN_DATALOADER, desc=f"{model_name} 에포크 {epoch + 1} - 학습 중", leave=False):
            images = images.to(DEVICE)
            classes = classes.to(DEVICE)

            output = model(images)
            loss = criterion(output, classes)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_cost += loss.item()

        train_cost = train_cost / len(TRAIN_DATALOADER)
        print(f'에포크 : {epoch + 1}, 학습 손실 : {train_cost:.3f}')

        # 검증 단계 및 메트릭 계산
        with torch.no_grad(): # 그래디언트 계산 비활성화
            model.eval() # 평가 모드
            val_loss = 0.0
            all_predicted_classes = []
            all_true_classes = []

            for images, classes in tqdm(VAILD_DATALOADER, desc=f"{model_name} 에포크 {epoch + 1} - 검증 중", leave=False):
                images = images.to(DEVICE)
                classes = classes.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, classes) # 검증 손실 계산
                val_loss += loss.item()

                # 메트릭 계산을 위해 예측 저장
                probs = F.softmax(outputs, dim=-1)
                outputs_classes = torch.argmax(probs, dim=-1)
                all_true_classes.extend(classes.cpu().numpy())
                all_predicted_classes.extend(outputs_classes.cpu().numpy())

            val_loss = val_loss / len(VAILD_DATALOADER) # 에포크 평균 검증 손실 계산

            # 에포크 전체에 대한 메트릭 계산
            accuracy = accuracy_score(all_true_classes, all_predicted_classes)
            # average='weighted' 또는 'macro', 'micro' 선택. 클래스 불균형 시 'weighted'가 일반적.
            precision = precision_score(all_true_classes, all_predicted_classes, average='weighted', zero_division=0)
            f1 = f1_score(all_true_classes, all_predicted_classes, average='weighted', zero_division=0)

            print(f'에포크 : {epoch + 1}, 검증 손실 : {val_loss:.3f}')
            print(f"에포크 : {epoch + 1}, 정확도 : {accuracy:.4f}")
            print(f"에포크 : {epoch + 1}, 정밀도 : {precision:.4f}")
            print(f"에포크 : {epoch + 1}, F1 점수 : {f1:.4f}")

            # 스케줄러 step 호출: 검증 손실을 기준으로 학습률 조정 판단
            scheduler.step(val_loss)

            # 조기 종료 및 베스트 모델 저장 로직
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_val_accuracy = accuracy # 최고 손실 시점의 정확도 저장
                patience_counter = 0 # 손실 개선 시 카운터 초기화
                # 현재 모델의 state_dict를 베스트 모델 상태로 저장 (깊은 복사)
                best_model_state = copy.deepcopy(model.state_dict())
                best_epoch = epoch + 1
                print(f'{model_name}: 검증 손실 개선 ({best_val_loss:.3f}). 모델 상태 저장 중.')
            else:
                patience_counter += 1 # 손실 개선 없음 -> 카운터 증가
                print(f'{model_name}: 검증 손실 개선 없음. Patience 카운터: {patience_counter}/{patience}')

            # 조기 종료 조건 확인
            if patience_counter >= patience:
                print(f'{model_name}: 조기 종료 발동. 총 {epoch + 1} 에포크 실행.')
                break # 현재 모델의 학습 루프 종료

    # --- 현재 모델의 학습 루프 종료 후 처리 ---
    # 가장 좋았던 모델 가중치를 로드하여 저장
    if best_model_state is not None:
        # 모델에 저장된 state_dict를 로드합니다. (조기 종료로 종료되지 않은 경우에도 최적 상태 로드)
        model.load_state_dict(best_model_state)
        # 모델 가중치 파일 경로 설정 (모델 이름 포함)
        model_save_path = f'./best_{model_name}_model.pt'
        torch.save(model.state_dict(), model_save_path)
        print(f'\n--- {model_name} 모델 학습 완료 ---')
        print(f'최고 검증 손실: {best_val_loss:.3f} (에포크 {best_epoch})')
        print(f'최고 손실 시점의 정확도: {best_val_accuracy:.4f}')
        print(f'최고 성능 모델 가중치를 {model_save_path} 에 저장했습니다.')
    else:
        print(f"\n--- {model_name} 모델 학습 완료 ---")
        print("모델 상태가 저장되지 않았습니다 (학습이 너무 일찍 실패했거나 손실이 개선되지 않았을 수 있습니다).")

    print(f"\n--- {model_name} 모델 학습 종료 ---\n")

# 모든 모델 학습 완료
print("===== 모든 VGG 모델 학습 완료 =====")


--- VGG11 모델 학습 시작 ---
VGG11 모델의 최종 분류 레이어를 4096 -> 3 으로 수정했습니다.
VGG11 모델 학습 시작 (최대 10 에포크, patience=3)


에포크 : 1, 학습 손실 : 1.149


에포크 : 1, 검증 손실 : 0.885
에포크 : 1, 정확도 : 0.6597
에포크 : 1, 정밀도 : 0.6849
에포크 : 1, F1 점수 : 0.6553
VGG11: 검증 손실 개선 (0.885). 모델 상태 저장 중.


에포크 : 2, 학습 손실 : 1.095


에포크 : 2, 검증 손실 : 1.111
에포크 : 2, 정확도 : 0.3229
에포크 : 2, 정밀도 : 0.1043
에포크 : 2, F1 점수 : 0.1576
VGG11: 검증 손실 개선 없음. Patience 카운터: 1/3


...

에포크 : 4, 검증 손실 : 1.104
에포크 : 4, 정확도 : 0.3403
에포크 : 4, 정밀도 : 0.1158
에포크 : 4, F1 점수 : 0.1728
VGG19: 검증 손실 개선 없음. Patience 카운터: 1/3


에포크 : 5, 학습 손실 : 1.103


에포크 : 5, 검증 손실 : 1.100
에포크 : 5, 정확도 : 0.3403
에포크 : 5, 정밀도 : 0.1158
에포크 : 5, F1 점수 : 0.1728
VGG19: 검증 손실 개선 없음. Patience 카운터: 2/3


에포크 : 6, 학습 손실 : 1.103


에포크 : 6, 검증 손실 : 1.100
에포크 : 6, 정확도 : 0.3438
에포크 : 6, 정밀도 : 0.1182
에포크 : 6, F1 점수 : 0.1759
VGG19: 검증 손실 개선 없음. Patience 카운터: 3/3
VGG19: 조기 종료 발동. 총 6 에포크 실행.

--- VGG19 모델 학습 완료 ---
최고 검증 손실: 1.099 (에포크 3)
최고 손실 시점의 정확도: 0.3299
최고 성능 모델 가중치를 ./best_VGG19_model.pt 에 저장했습니다.

--- VGG19 모델 학습 종료 ---

===== 모든 VGG 모델 학습 완료 =====
